<a href="https://colab.research.google.com/github/YongweiChe/motion_sensor/blob/master/CounterPointChecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bash Terminal Commands

In [12]:
!pip install thinkx
!pip install pydub
!pip install mido
!pip install pygame
!git clone https://github.com/beloiual/counterpoint-checker #forks the repository with the wavfile data


fatal: destination path 'counterpoint-checker' already exists and is not an empty directory.


In [13]:
from collections import namedtuple
import wave
import math
import plotly.express as px
from pydub import AudioSegment
import IPython


Functions that check for length constraints

In [14]:
def check_same_length(list1, list2):
  return len(list1) == len(list2)

def check_length(list1):
  return list1 <= 16 and list1 >= 8 #cantus firmus and counterpoint must be between 8 and 16 notes

Check for Miscellenious constraints

In [15]:
def check_first(above_check, list1, list2):
  above_consonance = [0, 7, 12]
  below_consonance = [0, 12]
  if (above_check == 1):
    return abs(list1[0] - list2[0]) in above_consonance
  else:
    return abs(list1[0] - list2[0]) in below_consonance

def check_last(list1, list2):
  consonance = [0, 12]
  last = len(list1) - 1
  return abs(list1[0] - list2[0]) in consonance

Check for correct ranges

In [17]:
def check_range(list1):
  return max(list1) - min(list1) > 16 #max difference between highest and lowest notes is a 10th (16 midi values)

def check_voice_cross(above_or_below, cantusf, counterp): # when above_or_below = 1, counterpoint above cantus firmus
  correct = True
  wrong_notes = []

  if (above_or_below == 1):
    for i in range(len(cantusf)):
      if (counterp[i] < cantusf[i]):
        wrong_notes.append(i)
        correct = False
  else:
    for i in range(len(cantusf)):
      if (counterp[i] > cantusf[i]):
        wrong_notes.append(i)
        correct = False
  return correct, wrong_notes

def check_voice_overlap(above_or_below, cantusf, counterp): # when above_or_below = 1, counterpoint above cantus firmus
  correct = True
  wrong_notes = []
  
  if (above_or_below == 1):
    for i in range(len(cantusf) - 1):
      if (counterp[i] < cantusf[i + 1] and counterp[i] >= cantusf[i]):
        wrong_notes.append(i)
        correct = False
  else:
    for i in range(len(cantusf) - 1):
      if (counterp[i] > cantusf[i + 1] and counterp[i] <= cantusf[i]):
        wrong_notes.append(i)
        correct = False
  return correct, wrong_notes

Functions that check note pairs for consonance

In [18]:
def check_harmonic_consonance(note1, note2):
  consonances = [0, 3, 4, 7, 8, 9] #list of consonant values
  return (abs((note1 - note2) % 12)) in consonances

def check_melodic_consonance(note1, note2):
  consonances = [0, 1, 2, 3, 4, 5, 7, 8, 9, 12] #list of consonant values
  interval = abs(note1 - note2)
  
  if interval <= 12:
    return interval in consonances
  else: 
    print("Leap too large")
    return False

Functions that check for peak constraints

In [19]:
def find_peak_cf(list_in):
  return list_in.index(max(list_in))

def find_peak_cp(list_in):
  orientation_val = 0 
  start = list_in[0]
# checks it the counterpoint is orientated up or down
  for i in range(len(list_in)): 
    orientation_val += list_in[i] - start
# depending on the orientation value find peak or trough
  if orientation_val >= 0:
    return list_in.index(max(list_in))
  else:
    return list_in.index(min(list_in))


def check_one_peak(list_in, peak_index):
  max = list_in[peak_index]
  if list_in.count(max) == 1:
    return True
  else:
    return False

Checking Harmonic and Melodic Consonance in the entire Counterpoint

In [20]:
def check_harmonic_consonance_in_list(list1, list2):
  # cycles through entire list to see if the two lists are harmonically consonant
  harmonic = True
  wrong_notes = []
  index = 0

  for x, y in zip(list1, list2):
    index += 1
    if not check_harmonic_consonance(x, y):
      harmonic = False
      wrong_notes.append(index)
  return harmonic, wrong_notes


def check_melodic_consonance_in_list(list1):
  # cycles through a list to check melodic consonant
  melodic = True
  wrong_notes = []

  for i in range(len(list1) - 1):
    if not check_melodic_consonance(list1[i], list1[i + 1]):
      print("Melodic error at ", i)
      melodic = False
      wrong_notes.append(i)
  return melodic, wrong_notes



In [21]:
def setKeyCenter(input_string):
  # gets key name
  keyName = input_string[0]

  STARTING_NOTE = 36
  SCALE_DEGREE = 1
  if (keyName == 'C'):
    STARTING_NOTE += 0
  elif (keyName == 'D'):
    STARTING_NOTE += 2
  elif (keyName == 'E'):
    STARTING_NOTE += 4
  elif (keyName == 'F'):
    STARTING_NOTE += 5
  elif (keyName == 'G'):
    STARTING_NOTE += 7
  elif (keyName == 'A'):
    STARTING_NOTE += 9
  elif (keyName == 'B'):
    STARTING_NOTE += 11
  else:
    raise Exception(keyName + " is not a valid note on the keyboard")

  # modifies starting note to account for accidentals
  if (len(input_string) == 5):
    if (input_string[1] == '#'):
      STARTING_NOTE += 1
    elif (input_string[1] == 'b'):
      STARTING_NOTE -= 1
    else:
      raise Exception(input_string[1] + " is not a valid accidental")
  
  # checks if major or minor
  isMajor = False
  if (input_string[-3:] == "maj"):
    isMajor = True

  # step movements between scale degrees for major and minor
  maj = [2, 2, 1, 2, 2, 2, 1]
  min = [2, 1, 2, 2, 1, 2, 2]

  keyCenter = {}

  # FILLS IN DICTIONARY
  if (isMajor):
    m = STARTING_NOTE
    i = SCALE_DEGREE
    # fills in dictionary from scale degree 1 upwards
    while (m < 128):
      keyCenter[m] = i
      m = m + maj[(i - 1) % 7]
      i += 1

    m = STARTING_NOTE
    i = SCALE_DEGREE
    # fills in dictionary from scale degree 1 downwards
    while (m > 0):

      keyCenter[m] = i
      m = m - maj[(i + 5) % 7]
      i -= 1
  else:
    m = STARTING_NOTE
    i = SCALE_DEGREE
    # fills in dictionary from scale degree 1 upwards
    while (m < 128):
      keyCenter[m] = i
      if ((i % 7 == 6 or i % 7 == 0) and m + 1 < 128):
        keyCenter[m + 1] = i
      m = m + min[(i - 1) % 7]
      i += 1

    m = STARTING_NOTE
    i = SCALE_DEGREE
    # fills in dictionary from scale degree 1 downwards
    while (m >= 0):
      keyCenter[m] = i
      if ((i % 7 == 6 or i % 7 == 0) and m + 1 >= 0):
        keyCenter[m + 1] = i
      m = m - min[(i + 5) % 7]
      i -= 1
  return keyCenter

Linear Regression

In [22]:
def isLinear(list_in,typeOfMusic):
  useMin = False
  useMax = False
  max = list_in[0]
  maxIndex=0
  minIndex=0
  min = list_in[0]
  count=0
  Index=0
  minCount=0
  maxCount=0

  if typeOfMusic=="Counter":
    Index=find_peak_cp(list_in)
  elif typeOfMusic=="Cantus":
    Index=find_peak_cf(list_in) #finds the index of the peak
  
  for j in range(Index+1): #counts the amount of indexs from the start to the max/min 
    count=count+1
  meanX=0
  meanY=0
  mTop=0
  mBottom=0
  for j in range(Index+1): #calculates the meanx by adding up the amount of steps and dividing by count for meanx and finding the average of each value in the list in a range
    meanY=meanY+list_in[j]
    meanX=meanX+j
  meanY=meanY/count
  meanX=meanX/count
 
  for j in range(Index+1): #calculates slope
    mTop=mTop+(j-meanX)*(list_in[j]-meanY)
    mBottom= mBottom + (j-meanX)*(j-meanX)
  

  
  if mBottom==0: #if the bottom is 0, will produce a false
    return "Random"
  m1=mTop/mBottom
  
  b=meanY-m1*meanX #calculates b
  


  if m1!=0:
    rsquared1Top=0
    rsquared1Bottom=0
    for j in range(Index+1): #finds r^2
      yBestfit= m1*j+b
      regression=(list_in[j]-yBestfit) #this is the regression at each point
      rsquared1Top=rsquared1Top+(regression*regression)
      rsquared1Bottom=rsquared1Bottom+((list_in[j]-meanY)*(list_in[j]-meanY))
    rsquared1=1-((rsquared1Top)/(rsquared1Bottom))
  else:
    rsquared1=1.0

  
  ###this does the same as the first but for the second half
  Index=Index+1 #changes it so we do not account for the peak (can be modified)
  count=len(list_in)-Index
  
  meanX=1
  meanY=0
  mTop=0
  mBottom=0

  if count!=0:
    for j in range(count):
      meanY=meanY+list_in[j+Index]
      meanX=meanX+j
    meanY=meanY/count
    meanX=meanX/(len(list_in)-count)
    
    for j in range(count):
      mTop=mTop+(j-meanX)*(list_in[j+Index]-meanY)
      mBottom= mBottom + (j-meanX)*(j-meanX)

    if mBottom==0:
      return "Random"
    m2=mTop/mBottom
    b=meanY-m2*meanX
    if m2!=0:
      rsquared2Top=0
      rsquared2Bottom=0
      for j in range(count):
        yBestfit= m2*j+b
        regression=(list_in[j+Index]-yBestfit)
        rsquared2Top=rsquared2Top+(regression*regression)
        rsquared2Bottom=rsquared2Bottom+((list_in[j+Index]-meanY)*(list_in[j+Index]-meanY))
      rsquared2=1-((rsquared2Top)/(rsquared2Bottom))
    else:
      rsquared2=1.0


    stringOutput= "Linear Threshold for Each Side of the Peak = " + str(round(rsquared1,3))+ " and "+ str(round(rsquared2,3))

  else:
    rsquared2=1
    stringOutput= "Linear Threshold for Each Side of the Peak = " + str(round(rsquared1,3))+ " and None"

  

  if (rsquared1==1) & (rsquared2==1) : #checks if a list of notes with a peak has a linear regression threshold of 100
    return "Pefectly Linear", stringOutput #outputs two strings
  elif (rsquared1>=0.9) & (rsquared2>=0.9) : #checks if a list of notes with a peak has a linear regression threshold of 90
    return "Very Smooth", stringOutput
  elif (rsquared1>=0.7) & (rsquared2>=0.7): #checks if a list of notes with a peak has a linear regression threshold of 70
    return "Relatively Smooth", stringOutput
  elif (rsquared1>=0.5) & (rsquared2>=0.5): #checks if a list of notes with a peak has a linear regression threshold of 50
    return "General Trend", stringOutput
  else:
    return "Random", stringOutput

Checking Motion

In [23]:
def motion(list1, list2): #input the list of values post key change
  motion=[]
  change1=[]
  change2=[]
  for i in range(len(list1)-1): #finds the values of change for the lists
    change1.append(list1[i+1]-list1[i])
  for i in range(len(list2)-1):
    change2.append(list2[i+1]-list2[i])
  for i in range(len(change1)):
    if i<len(change2): #checks to make sure that there are no hanging notes
      if change1[i]==change2[i]: #parallel check (checks if both jump the same amount)
        motion.append("Parallel") 
      elif ((change1[i]>0) & (change2[i]>0)) | ((change1[i]<0) & (change2[i]<0)) | ((change1[i]==0) & (change2[i]==0)): #similar check (checks if both are in the same direction)
        motion.append("Similar")
      elif ((change1[i]!=0) & (change2[i]==0)) | ((change1[i]==0) & (change2[i]!=0)): #oblique check (checks if one stays the same and the other changes direction)
        motion.append("Oblique")
      elif ((change1[i]>0) & (change2[i]<0)) | ((change1[i]<0) & (change2[i]>0)): #contrary check (checks if one goes up and the other goes down)
        motion.append("Contrary")
    else:
      motion.append("Hanging Note")

  
  return motion    


In [24]:
def motion3InARow(motion): #Checks if motion occures three times in a row
  no_3_in_row = True
  wrong_notes = []

  for i in range(len(motion)-2):
    if motion[i] == motion[i+1]:
      if motion[i+1]==motion[i+2]:
        no_3_in_row = False
        wrong_notes.append(i + 2)
  return no_3_in_row, wrong_notes

# returns whether correct or not and a list of the locations of inaccuracies
def checkParallelFifthsOctaves(list1, list2):
  correct = True
  occurances = []
  for i in range(len(list1) - 1):
    if (list1[i] - list1[i + 1] == list2[i] - list2[i + 1]):
      if (abs(list1[i] - list2[i]) % 12 == 0 or abs(list1[i] - list2[i]) % 12 == 7):
        occurances.append(i)
        correct = False
  return correct, occurances

# returns whether correct or not and a list of the locations of inaccuracies
def checkHiddenFifthsOctaves(list1, list2):
  correct = True
  occurances = []
  for i in range(len(list1) - 1):
    if ((list1[i] - list1[i + 1] > 0 and list2[i] - list2[i + 1] > 0) or (list1[i] - list1[i + 1] < 0 and list2[i] - list2[i + 1] < 0)):
      if (abs(list1[i + 1] - list2[i + 1]) % 12 == 0 and abs(list1[i] - list2[i]) % 12 != 0):
        occurances.append(i)
        correct = False
      if (abs(list1[i + 1] - list2[i + 1]) % 12 == 7 and abs(list1[i] - list2[i]) % 12 != 7):
        occurances.append(i)
        correct = False
  return correct, occurances

# returns whether correct or not and a list of the locations of inaccuracies
def check_7th_resolution(list1): 
  resolution_good = True
  wrong_notes = []

  for i in range(len(list1) - 1):
    if (list1[i] % 8 == 7):
      if (list1[i] % 8 != 0):
        resolution_good = False
        wrong_notes.append(i)
  return resolution_good, wrong_notes

# returns whether correct or not and a list of the locations of inaccuracies
def check_leaps_resolve_opposite(list1):
  leaps_good = True
  wrong_notes = []

  for i in range(len(list1) - 2):
    interval = list1[i + 1] - list1[i]
    interval_2 = list1[i + 2] - list1[i + 1]
    if (abs(interval) > 1):
      if (interval * interval_2 > 0):
        leaps_good = False
        wrong_notes.append[i + 1]

  return leaps_good, wrong_notes



Inputing a MIDI file and converting it to an array of MIDI Numbers

In [25]:
from mido import MidiFile #import the mido library
def midiInput(mid):
  allnotes=[]
  cantusNotes=[]
  counterNotes=[]

  mid = MidiFile('counterpoint-checker/MidiTest/mid.mid') #inputs a MIDI file into the function

  for i, track in enumerate(mid.tracks): #for every i within each track in a midi track list, this will check each message in the "note_on" section in each track for a MIDI number
      #print('Track {}: {}'.format(i, track.name)) ##for testing
      for msg in track:
        if msg.type == 'note_on':
          #print(msg.note) ##for testing
          allnotes.append(msg.note) #appends the note number onto an array


  for i in range(len(allnotes)): #splits the notes between track 1 and 2
    if i<(len(allnotes)/2):
      cantusNotes.append(allnotes[i])
    else:
      counterNotes.append(allnotes[i])                ####here there is an error in counting, need to ask

  return cantusNotes,counterNotes #returns an array for the cantus firmus and counterpoint


  

Reads the list of provided wav files and appends them together into a file called "cantus.wav"

In [26]:
def cantusWavCreate(infilesCantius):
  outfileCantus = "cantus.wav"


  dataCantus= []

  for infileC in infilesCantius: 
      
      waveC = wave.open(infileC, 'rb')

      dataCantus.append( [waveC.getparams(), waveC.readframes(waveC.getnframes())] )
      

      waveC.close()

  outputCantus = wave.open(outfileCantus, 'wb')

  outputCantus.setparams(dataCantus[0][0])
  i=0
  for i in range(len(dataCantus)):
    outputCantus.writeframes(dataCantus[i][1])
  outputCantus.close()
  return outfileCantus

Reads the list of provided wav files and appends them together into a file called "counter.wav"

In [27]:
def counterWavCreate(infilesCounter):
  outfileCounter = "counter.wav"


  dataCounter= []

  for infileCount in infilesCounter:

      waveCounter = wave.open(infileCount, 'rb')

      dataCounter.append( [waveCounter.getparams(), waveCounter.readframes(waveCounter.getnframes())] )
      

      waveCounter.close()

  outputCounter = wave.open(outfileCounter, 'wb')

  outputCounter.setparams(dataCounter[0][0])
  i=0
  for i in range(len(dataCounter)):
    outputCounter.writeframes(dataCounter[i][1])
  outputCounter.close()

  return outfileCounter

Layers the counterpoint and cantus firmus wav files together to playback some notes.

In [28]:
def playAudio(outfileCantus,outfileCounter):
  sound1 = AudioSegment.from_file(outfileCantus)
  sound2 = AudioSegment.from_file(outfileCounter)

  combined = sound1.overlay(sound2)

  combined.export("counterpointChecker.wav", format='wav')

  IPython.display.Audio('counterpointChecker.wav')

Compiling .wav files into an array

In [35]:
def wavWork(cantus_firmus,counterpoint): #input the array of note values and counterpoint values (as a string)
  infilesCantius=[] 
  i=0
  for i in range(len(cantus_firmus)): #checks each possible note in the cantus and inputs a wav file for that note into an array
    infilesCantius.append('counterpoint-checker/WaveFiles/' + cantus_firmus[i] + '.wav')

  infilesCounter=[]
  i=0
  for i in range(len(counterpoint)): #checks each possible note in the counterpoint and inputs a wav file for that note into an array 
    infilesCounter.append('counterpoint-checker/WaveFiles/' + cantus_firmus[i] + '.wav')

  outfileCantus=cantusWavCreate(infilesCantius)
  outfileCounter=counterWavCreate(infilesCounter)
  playAudio(outfileCantus,outfileCounter)


In [30]:
def allNotesInKey(cantus_firmus_int,cantus_fimus, music_note_to_intKEY):
  notCorrect = False
  for i in range(len(cantus_firmus_int)): #checks if all notes are in the key
      if cantus_firmus_int[i] in music_note_to_intKEY:
        i=i
      else:
        cantus_firmus_KEY.append(cantus_firmus[i])
        notCorrect=True
        
  if notCorrect==False:
    return True
      #print("All notes are in the Key", musickey, "for the Cantus Firmus") 
  elif notCorrect == True:
    return False
      #print("Notes", cantus_firmus_KEY,"are not in the key",musickey,"for the Cantus Firmus")

The "Main()" Function of the Program

In [32]:
music_note_to_int1 = {'C2': 36, 'C#2': 37, 'Db2': 37,'D2': 38, 'D#2': 39, 'Eb2': 39, 'E2': 40, 'F2': 41, 'F#2': 42, 'Gb2': 42, 'G2': 43, 'G#2': 44, 'Ab2': 44, 'A2': 45, 'A#2': 46, 'Bb2': 46, 'B2': 47}
music_note_to_int2 = {'C3': 48, 'C#3': 49, 'Db3': 49,'D3': 50, 'D#3': 51, 'Eb3': 51, 'E3': 52, 'F3': 53, 'F#3': 54, 'Gb3': 54, 'G3': 55, 'G#3': 56, 'Ab3': 56, 'A3': 57, 'A#3': 58, 'Bb3': 58, 'B3': 59}
music_note_to_int3 = {'C4': 60, 'C#4': 61, 'Db4': 61,'D4': 62, 'D#4': 63, 'Eb4': 63, 'E4': 64, 'F4': 65, 'F#4': 66, 'Gb4': 66, 'G4': 67, 'G#4': 68, 'Ab4': 68, 'A4': 69, 'A#4': 70, 'Bb4': 70, 'B4': 71}
music_note_to_int4 = {'C5': 72, 'C#5': 73, 'Db5': 73,'D5': 74, 'D#5': 75, 'Eb5': 75, 'E5': 76, 'F5': 77, 'F#5': 78, 'Gb5': 78, 'G5': 79, 'G#5': 80, 'Ab5': 80, 'A5': 81}
music_note_to_int1.update(music_note_to_int2)
music_note_to_int1.update(music_note_to_int3)
music_note_to_int1.update(music_note_to_int4)
notCorrect=False

print("Cantus firmus input: ")
#cantus_firmus_in = input() #input the cantus firmus
cantus_firmus_in = "C2 D2 E2 G2 A2 F2 D2 C2" #test
cantus_firmus = cantus_firmus_in.split()
cantus_firmus_int = []
cantus_firmus_int_KEY = []

print("Counterpoint input: ")
#counterpoint_in = input() #input the counterpoint
counterpoint_in = "C3 B2 A2 G2 F2 E2 D2 C2" #test
counterpoint = counterpoint_in.split()
counterpoint_int = []
counterpoint_int_KEY = []

print("Counterpoint above or below: ")
#aorb_in = input() #input a or b
aorb_in = 'a' #test
aorb_int = 0
if (aorb_in == 'a'):
    aorb_int = 1
elif (aorb_in == 'b'):
    aorb_int = -1
else:
    print("Invalid input")

print("Input Key: ")
#musickey=input() #input the key name
musickey = "Cmaj" #test
music_note_to_intKEY = setKeyCenter(musickey)

cantus_firmus_KEY=[]
counterpoint_KEY=[]


if (check_same_length(cantus_firmus, counterpoint)): #checks if they are the same length
  for i in cantus_firmus: #checks the inputed notes and iterates through it
    cantus_firmus_int.append(music_note_to_int1[i]) #appends to an int array the midi value of the note
  for i in counterpoint:
    counterpoint_int.append(music_note_to_int1[i])

for i in cantus_firmus_int:
  cantus_firmus_int_KEY.append(music_note_to_intKEY[i])
for i in counterpoint_int:
  counterpoint_int_KEY.append(music_note_to_intKEY[i])


checkA, listA = check_harmonic_consonance_in_list(cantus_firmus_int, counterpoint_int)
checkB, listB = check_melodic_consonance_in_list(cantus_firmus_int) and check_melodic_consonance_in_list(counterpoint_int) #tests
checkRangeCF = check_range(cantus_firmus_int)
checkRangeCP = check_range(counterpoint_int)
#print(find_peak_cp(counterpoint_int))
peakCF_index = find_peak_cf(cantus_firmus_int)
peakCP_index = find_peak_cp(counterpoint_int)
check_one_peakCF = check_one_peak(cantus_firmus_int, peakCF_index)
check_one_peakCP = check_one_peak(counterpoint_int, peakCP_index)
checkPeakNotSame = peakCF_index != peakCP_index
checkC1 = isLinear(counterpoint_int, "Counter")
checkC2 = isLinear(cantus_firmus_int, "Cantus")
checkM = motion(cantus_firmus_int_KEY, counterpoint_int_KEY)
checkMotion3, listM3 = motion3InARow(checkM)
checkPerfect5ths, listP5 = checkParallelFifthsOctaves(cantus_firmus_int, counterpoint_int)
checkHidden5ths, listH5 = checkHiddenFifthsOctaves(cantus_firmus_int, counterpoint_int)
checkK1 = allNotesInKey(cantus_firmus_int,cantus_firmus, music_note_to_intKEY)
checkK2 = allNotesInKey(counterpoint_int,counterpoint, music_note_to_intKEY)
check7th_resolveCF, list7thCF = check_7th_resolution(cantus_firmus_int_KEY)
check7th_resolveCP, list7thCP = check_7th_resolution(counterpoint_int_KEY)
checkLeapsOppositeCF, listLO1 = check_leaps_resolve_opposite(cantus_firmus_int_KEY)
checkLeapsOppositeCP, listLO2 = check_leaps_resolve_opposite(counterpoint_int_KEY)
checkVC, listVC = check_voice_cross(aorb_int, cantus_firmus_int, counterpoint_int)
checkVO, listVO = check_voice_overlap(aorb_int, cantus_firmus_int, counterpoint_int)
checkFirst = check_first(aorb_int, counterpoint_int, cantus_firmus_int)
checkLast = check_last(counterpoint_int, cantus_firmus_int)


print("Harmonically good:", checkA, listA)
print("Melodically good:", checkB, listB)
print("Ranges good: CF: ", checkRangeCF, "CP: ", checkRangeCP)
print("One peak: CF: ", check_one_peakCF, "CP: ", check_one_peakCP)
print("Peaks are not at the same place: ", checkPeakNotSame)
print("Cantus Firmus notes in key",musickey,":",checkK1)
print("CounterPoint notes in key",musickey,":",checkK2)
print("Cantus Firmus Linear:", checkC2)
print("CounterPoint Linear:", checkC1)
print("Motion of the Music: ", checkM)
print("No Three Parallel Motion in a Row: ", checkMotion3, listM3)
print("Perfect Fifths & Octaves: ", checkPerfect5ths, listP5)
print("Hidden Fifths & Octaves: ", checkHidden5ths, listH5)
print("7ths resolve: CF: ", check7th_resolveCF, "CP: ", check7th_resolveCP)
print("Leaps are followed by opposite direction: CF:", checkLeapsOppositeCF, listLO1, "CP: ", checkLeapsOppositeCP, listLO2)
print("No voice-crossing: ", checkVC, listVC)
print("No voice-overlap: ", checkVO, listVO)
print("First interval is good: ", checkFirst)
print("Last interval is good: ", checkLast)

wavWork(cantus_firmus,counterpoint)
sound1 = AudioSegment.from_file("cantus.wav")
sound2 = AudioSegment.from_file("counter.wav")

combined = sound1.overlay(sound2)

combined.export("counterpointChecker.wav", format='wav')

IPython.display.Audio('counterpointChecker.wav')



Cantus firmus input: 
Counterpoint input: 
Counterpoint above or below: 
Input Key: 


TypeError: ignored

Test Case Copy and Paste:

C2 D2 E2 F2 G2 A2 B2 C3

C3 B2 A2 G2 F2 E2 D2 C2


